In [1]:
#!pip install fasttext
#!rm -rf SaRaH/
#!git clone https://github.com/alessandrocuda/SaRaH
#!wget http://www.italianlp.it/twitter128.bin

In [3]:
import tensorflow as tf
#import tensorflow_addons as tfa

from tensorflow.keras import backend as K

#from tensorflow.keras.engine import Layer, InputSpec, InputLayer

#models
from tensorflow.keras.models import Model, Sequential
#layers
from tensorflow.keras.layers import Dropout, Embedding, concatenate
from tensorflow.keras.layers import Conv1D, MaxPool1D, Conv2D, MaxPool2D, ZeroPadding1D, GlobalMaxPool1D, GlobalAveragePooling1D
from tensorflow.keras.layers import Dense, Input, Flatten, BatchNormalization, Activation
from tensorflow.keras.layers import Concatenate, Dot, Concatenate, Multiply, RepeatVector
from tensorflow.keras.layers import Bidirectional, TimeDistributed
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Lambda, Permute
#training
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

#model_selection_evaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score

#word2vec
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from gensim.test.utils import datapath
#import fasttext.util

#utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys

#src
#root_project = "/content/SaRaH/"
root_project = "/Users/Alessandro/Dev/repos/SaRaH/"
#root_project = "/home/jupyter/SaRaH/"
sys.path.append(root_project)
from src.data.utils import load_csv_to_dict, string_to_list, set_unkmark
from src.features.word_embedding import get_index_key_association, get_int_seq, build_keras_embedding_matrix, get_data_to_emb

%load_ext autoreload
%autoreload 2

## System check

In [5]:
print("TF version: {}".format(tf.__version__))

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    print('GPU device not found')

TF version: 2.4.1
GPU device not found


# Path

In [6]:
dataset_dev_path           = root_project + "dataset/haspeede2/preprocessed2/dev/dev.csv"
dataset_test_tweets_path   = root_project + "dataset/haspeede2/preprocessed2/reference/reference_tweets.csv"
w2v_path                   = root_project + "results/model/word2vec/word2vec2.wordvectors"

In [ ]:
#load w2v and data
w2v = KeyedVectors.load(w2v_path)